In [1]:
from notebook_utils import setup
import pandas as pd
import networkx as nx
from collections import defaultdict

setup()

import matplotlib.style as style

style.use(["seaborn-white", "seaborn-paper"])

In [2]:
DATE = "16-dec"
DATA_DIR = "../data/{}/".format(DATE)
EXPORT_DIR = "../data/dataframes/{}/".format(DATE)

In [2]:
df_recent_tweets = pd.read_pickle(EXPORT_DIR + 'df_recent_tweets.pickle')

NameError: name 'EXPORT_DIR' is not defined

In [4]:
df_retweets = pd.read_pickle(EXPORT_DIR + 'df_retweets.pickle')

In [4]:
df_users_final = pd.read_pickle("./df_users_final.pickle")

In [5]:
import pickle 
with open("./cached_graph.pickle", "rb") as f:
    cached_graph = pickle.load(f)

In [6]:
cached_graph.keys()

dict_keys(['all_user_clusters', 'graph', 'node_id_map', 'clustering_directed_unweighted', 'clustering_directed_weighted', 'clustering_undirected_unweighted'])

In [7]:
node_community_map = cached_graph["clustering_directed_weighted"][0]
node_id_map = cached_graph["node_id_map"]

In [8]:
import numpy as np
def get_cluster(user_id):
    if (user_id in node_id_map):
        node_id = node_id_map[user_id]
        if (node_id in node_community_map):
            cluster = node_community_map[node_id]
            if (cluster < 5):
                return cluster
    return np.nan


df_recent_tweets["cluster"] = df_recent_tweets.apply(lambda tweet: get_cluster(tweet["user"]), axis=1).astype('Int64')

In [9]:
df_retweets["cluster"] = df_retweets.apply(lambda tweet: get_cluster(tweet["user"]), axis=1).astype('Int64')
df_retweets.head()

,user,timestamp,retweeted,retweetedFrom_user,cluster
0,3932314393,2020-12-10T20:29:17Z,1337090636344827906,1097618307778150400,2
1,3932314393,2020-12-12T01:48:31Z,1337549060853665793,364214133,2
2,3932314393,2020-12-12T17:02:01Z,1337750414779944960,797409551473590272,2
3,3932314393,2020-12-12T17:48:21Z,1337790419875352576,25073877,2
4,3932314393,2020-12-15T03:08:36Z,1337935422450769921,4165642155,2


In [10]:
df_retweets.to_pickle("./df_retweets_with_cluster.pickle")
df_recent_tweets.to_pickle("./df_recent_tweets_with_cluster.pickle")

In [5]:
df_retweets = pd.read_pickle("./df_retweets_with_cluster.pickle")
df_recent_tweets =  pd.read_pickle("./df_recent_tweets_with_cluster.pickle")

In [7]:
df_users_final.head()

,created_at,friends_count,name,verified,followers_count,location,handle,data_source,active_status,cluster,l_closeness,r_closeness
id,,,,,,,,,,,,
3288305608,Mon May 18 11:23:04 +0000 2015,626,kastro J.B 🔴,False,157,"London, England",BeainiKastro,new,active,1,NaN,NaN
1333163755295924224,Sun Nov 29 21:39:55 +0000 2020,93,ConfederacyofDunces,False,9,"Nashville, TN",RobVM3,new,active,<NA>,NaN,NaN
1325878139537354752,Mon Nov 09 19:10:57 +0000 2020,135,KBR,False,18,,KBR02093679,new,active,<NA>,NaN,NaN
304050894,Mon May 23 21:07:18 +0000 2011,77,Giuseppe,False,69,Western Air Temple,PizzaEquality,new,active,0,NaN,NaN
1305445745328361472,Mon Sep 14 09:58:30 +0000 2020,22,Carolina Belardinelli,False,2,,CarolinaBelard2,new,active,1,NaN,NaN


## Annotate tweets with retweets/quotes in cluster

In [6]:
suspended_user_ids = set(df_users_final[df_users_final["active_status"] == "suspended"].index.values)
print(len(suspended_user_ids))

inactive_user_ids = set(df_users_final[df_users_final["active_status"] == "inactive"].index.values)
print(len(inactive_user_ids))

99884
115868


In [11]:
df_recent_tweets.head()

,datastore_id,urls,hasMedia,hashtags,retweet_count,quote_count,user,text,quote_tweet,timestamp,...,#nomailinvoting,#ilhanomarballotharvesting,vote by mail fraud,#mailinvoterfraud,#votebymailfraud,#ilhanomarvoterfraud,#stopgopvoterfraud,#discardedballots,#hackedvotingmachines,cluster
0,1327172028709801985,[https://twitter.com/i/web/status/132717202870...,False,"[auspol, ConcedeNowTrump]",0,0,2313341894,"@realDonaldTrump ""No evidence of US election f...",None,2020-11-13T08:50:43Z,...,0,0,0,0,0,0,0,0,0,0
1,1327172032002486273,[https://twitter.com/i/web/status/132717203200...,False,[],0,0,750121838,@TRUTHandFREED0M @kpolantz Plus the evidence o...,None,2020-11-13T08:50:44Z,...,0,0,0,0,0,0,0,0,0,1
2,1327172032061173761,[],False,[],0,0,546655747,Hear HEAR @RealDLHughley,1326904412527276037,2020-11-13T08:50:44Z,...,0,0,0,0,0,0,0,0,0,0
3,1327172033143316480,[https://www.bbc.co.uk/news/election-us-2020-5...,False,[],0,0,25709850,US election security officials reject Trump's ...,None,2020-11-13T08:50:45Z,...,0,0,0,0,0,0,0,0,0,0
4,1327172039669665794,[],True,[],0,0,1254057289113317382,The election results themselves are disputed b...,None,2020-11-13T08:50:46Z,...,0,0,0,0,0,0,0,0,0,2


In [7]:
quote_tweet_author_map = {}
quoted_tweet_ids = set(df_recent_tweets["quote_tweet"].values)

for i, tweet_id, user in df_recent_tweets[["datastore_id", "user"]].itertuples():
    if (tweet_id in quoted_tweet_ids):
        quote_tweet_author_map[tweet_id] = user

In [8]:
from collections import defaultdict

tweet_shares_map = defaultdict(lambda: {
    "retweets_by_cluster": defaultdict(lambda: 0),
    "quotes_by_cluster": defaultdict(lambda: 0),
    "retweets_by_suspended": 0,
    "quotes_by_suspended": 0,
    "retweets_crawled": 0,
    "quotes_crawled": 0
})

user_shares_map = defaultdict(lambda: {
    "retweets_by_cluster": defaultdict(lambda: 0),
    "quotes_by_cluster": defaultdict(lambda: 0),
    "retweets_by_suspended": 0,
    "quotes_by_suspended": 0,
    "retweets_crawled": 0,
    "quotes_crawled": 0
})

def get_user_status(user_id):
    if user_id in inactive_user_ids:
        return 'inactive'
    elif user_id in suspended_user_ids:
        return 'suspended'
    else:
        return 'active'

retweet_author_status = []
tweet_author_status = []


for i, user_id, user_cluster, retweeted_tweet_id, retweeted_tweet_author in df_retweets[["user", "cluster", "retweeted", "retweetedFrom_user"]].itertuples():
    if (pd.notna(user_cluster)):
        tweet_shares_map[retweeted_tweet_id]["retweets_by_cluster"][user_cluster] += 1
        user_shares_map[retweeted_tweet_author]["retweets_by_cluster"][user_cluster] += 1
    user_status = get_user_status(user_id)
    retweet_author_status.append(user_status)
    if (user_status == 'suspended'):
        retweet_author_status.append(user_status)
        tweet_shares_map[retweeted_tweet_id]["retweets_by_suspended"] += 1
        user_shares_map[retweeted_tweet_author]["retweets_by_suspended"] += 1
    tweet_shares_map[retweeted_tweet_id]["retweets_crawled"] += 1
    user_shares_map[retweeted_tweet_author]["retweets_crawled"] += 1
    if (i % 100000 == 0):
        print("Processed {} retweets".format(i))
print("Done processing retweets")

for i, user_id, quoted_tweet_id, cluster in df_recent_tweets[["user", "quote_tweet", "cluster"]].itertuples():
    user_status = get_user_status(user_id)
    tweet_author_status.append(user_status)
    quote_tweet_author = quoted_tweet_id in quote_tweet_author_map and quote_tweet_author_map[quoted_tweet_id]
    if (quoted_tweet_id is not None):
        if (pd.notna(cluster)):
            tweet_shares_map[quoted_tweet_id]["quotes_by_cluster"][cluster] += 1
            if (quote_tweet_author):
                user_shares_map[quote_tweet_author]["quotes_by_cluster"][cluster] += 1

        if (user_status == "suspended"):
            tweet_shares_map[quoted_tweet_id]["quotes_by_suspended"] += 1

            if (quote_tweet_author):
                user_shares_map[quote_tweet_author]["quotes_by_suspended"] += 1
    
    tweet_shares_map[quoted_tweet_id]["quotes_crawled"] += 1
    if (quote_tweet_author):
        user_shares_map[quote_tweet_author]["quotes_crawled"] += 1
    
    if (i % 100000 == 0):
        print("Processed {} tweets".format(i))
print("Done processing tweets")

Processed 0 retweets
Processed 100000 retweets
Processed 200000 retweets
Processed 300000 retweets
Processed 400000 retweets
Processed 500000 retweets
Processed 600000 retweets
Processed 700000 retweets
Processed 800000 retweets
Processed 900000 retweets
Processed 1000000 retweets
Processed 1100000 retweets
Processed 1200000 retweets
Processed 1300000 retweets
Processed 1400000 retweets
Processed 1500000 retweets
Processed 1600000 retweets
Processed 1700000 retweets
Processed 1800000 retweets
Processed 1900000 retweets
Processed 2000000 retweets
Processed 2100000 retweets
Processed 2200000 retweets
Processed 2300000 retweets
Processed 2400000 retweets
Processed 2500000 retweets
Processed 2600000 retweets
Processed 2700000 retweets
Processed 2800000 retweets
Processed 2900000 retweets
Processed 3000000 retweets
Processed 3100000 retweets
Processed 3200000 retweets
Processed 3300000 retweets
Processed 3400000 retweets
Processed 3500000 retweets
Processed 3600000 retweets
Processed 370000

In [9]:
user_shares_map["25073877"]

{'retweets_by_cluster': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
             {2: 863596, 1: 292242, 4: 32117, 3: 37414, 0: 6257}),
 'quotes_by_cluster': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
             {0: 29941, 1: 26607, 2: 94552, 3: 3556, 4: 2326}),
 'retweets_by_suspended': 291398,
 'quotes_by_suspended': 28156,
 'retweets_crawled': 1560373,
 'quotes_crawled': 231142}

In [10]:
tweet_shares_map["1327038586819145732"]

{'retweets_by_cluster': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
             {0: 3352, 2: 58, 1: 13, 4: 5, 3: 2}),
 'quotes_by_cluster': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
             {0: 1640, 4: 4, 2: 37, 1: 7, 3: 1}),
 'retweets_by_suspended': 38,
 'quotes_by_suspended': 71,
 'retweets_crawled': 3441,
 'quotes_crawled': 2899}

In [11]:
N_CLUSTERS = 5

def get_user_shares_by_cluster (user_id, share_type, cluster):
    return user_shares_map[user_id][share_type][cluster]

def get_user_cols():
    retweet_by_cluster_cols = [[] for i in range(N_CLUSTERS)]
    quote_by_cluster_cols = [[] for i in range(N_CLUSTERS)]
    retweets_by_suspended = []
    quotes_by_suspended = []
    retweets_crawled = []
    quotes_crawled = []

    for user_id in df_users_final.index:
        retweets_by_suspended.append(user_shares_map[user_id]["retweets_by_suspended"])
        quotes_by_suspended.append(user_shares_map[user_id]["quotes_by_suspended"])
        retweets_crawled.append(user_shares_map[user_id]["retweets_crawled"])
        quotes_crawled.append(user_shares_map[user_id]["quotes_crawled"])
        for cluster in range(N_CLUSTERS):
            retweet_by_cluster_cols[cluster].append(get_user_shares_by_cluster(user_id, "retweets_by_cluster", cluster))
            quote_by_cluster_cols[cluster].append(get_user_shares_by_cluster(user_id, "quotes_by_cluster", cluster))
    return retweet_by_cluster_cols, quote_by_cluster_cols, retweets_by_suspended, quotes_by_suspended, retweets_crawled, quotes_crawled

In [12]:
user_cols = get_user_cols()

for c in range(N_CLUSTERS):
    print("Cluster", c)
    df_users_final["retweets_by_cluster_" + str(c)] = user_cols[0][c]
    df_users_final["quotes_by_cluster_" + str(c)] = user_cols[1][c]
print("Adding additional cols")
df_users_final["retweets_by_suspended"] = user_cols[2]
df_users_final["quotes_by_suspended"] = user_cols[3]
df_users_final["retweets_crawled"] = user_cols[4]
df_users_final["quotes_crawled"] = user_cols[5]

df_users_final.head()

Cluster 0
Cluster 1
Cluster 2
Cluster 3
Cluster 4
Adding additional cols


,created_at,friends_count,name,verified,followers_count,location,handle,data_source,active_status,cluster,...,retweets_by_cluster_2,quotes_by_cluster_2,retweets_by_cluster_3,quotes_by_cluster_3,retweets_by_cluster_4,quotes_by_cluster_4,retweets_by_suspended,quotes_by_suspended,retweets_crawled,quotes_crawled
id,,,,,,,,,,,,,,,,,,,,,
3288305608,Mon May 18 11:23:04 +0000 2015,626,kastro J.B 🔴,False,157,"London, England",BeainiKastro,new,active,1,...,0,0,0,0,0,0,0,0,0,0
1333163755295924224,Sun Nov 29 21:39:55 +0000 2020,93,ConfederacyofDunces,False,9,"Nashville, TN",RobVM3,new,active,<NA>,...,0,0,0,0,0,0,0,0,0,0
1325878139537354752,Mon Nov 09 19:10:57 +0000 2020,135,KBR,False,18,,KBR02093679,new,active,<NA>,...,0,0,0,0,0,0,0,0,0,0
304050894,Mon May 23 21:07:18 +0000 2011,77,Giuseppe,False,69,Western Air Temple,PizzaEquality,new,active,0,...,0,0,0,0,0,0,0,0,0,0
1305445745328361472,Mon Sep 14 09:58:30 +0000 2020,22,Carolina Belardinelli,False,2,,CarolinaBelard2,new,active,1,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df_users_final.loc["25073877"]

created_at                  2009-03-18T13:46:38Z
friends_count                                 50
name                             Donald J. Trump
verified                                    True
followers_count                         87364085
location                          Washington, DC
handle                           realDonaldTrump
data_source                                  old
active_status                          suspended
cluster                                        1
l_closeness                                  NaN
r_closeness                             0.454827
retweets_by_cluster_0                       6257
quotes_by_cluster_0                        29941
retweets_by_cluster_1                     292242
quotes_by_cluster_1                        26607
retweets_by_cluster_2                     863596
quotes_by_cluster_2                        94552
retweets_by_cluster_3                      37414
quotes_by_cluster_3                         3556
retweets_by_cluster_

In [27]:
df_users_final.to_pickle("./df_users_final_with_metrics.pickle")

In [3]:
df_users_final = pd.read_pickle("./df_users_final_with_metrics.pickle")

In [22]:
N_CLUSTERS = 5

def get_shares_by_cluster (tweet_id, share_type, cluster):
    return tweet_shares_map[tweet_id][share_type][cluster]


def get_cols():
    retweet_by_cluster_cols = [[] for i in range(N_CLUSTERS)]
    quote_by_cluster_cols = [[] for i in range(N_CLUSTERS)]
    retweets_by_suspended = []
    quotes_by_suspended = []
    retweets_crawled = []
    quotes_crawled = []

    for tweet_id in df_recent_tweets["datastore_id"]:
        retweets_by_suspended.append(tweet_shares_map[tweet_id]["retweets_by_suspended"])
        quotes_by_suspended.append(tweet_shares_map[tweet_id]["quotes_by_suspended"])
        retweets_crawled.append(tweet_shares_map[tweet_id]["retweets_crawled"])
        quotes_crawled.append(tweet_shares_map[tweet_id]["quotes_crawled"])
        for cluster in range(N_CLUSTERS):
            retweet_by_cluster_cols[cluster].append(get_shares_by_cluster(tweet_id, "retweets_by_cluster", cluster))
            quote_by_cluster_cols[cluster].append(get_shares_by_cluster(tweet_id, "quotes_by_cluster", cluster))
    return retweet_by_cluster_cols, quote_by_cluster_cols, retweets_by_suspended, quotes_by_suspended, retweets_crawled, quotes_crawled

In [23]:
retweets_by_cluster, quotes_by_cluster, retweets_by_suspended, quotes_by_suspended, retweets_crawled, quotes_crawled = get_cols()

In [24]:
for c in range(N_CLUSTERS):
    print("Cluster", c)
    df_recent_tweets["retweets_by_cluster_" + str(c)] = retweets_by_cluster[c]
    df_recent_tweets["quotes_by_cluster_" + str(c)] = quotes_by_cluster[c]
print("Adding additional cols")
df_recent_tweets["author_active_status"] = tweet_author_status
df_recent_tweets["retweets_by_suspended"] = retweets_by_suspended
df_recent_tweets["quotes_by_suspended"] = quotes_by_suspended
df_recent_tweets["retweets_crawled"] = retweets_crawled
df_recent_tweets["quotes_crawled"] = quotes_crawled

Cluster 0
Cluster 1
Cluster 2
Cluster 3
Cluster 4
Adding additional cols


In [29]:
df_recent_tweets[df_recent_tweets["user"] == "25073877"][:10]

,datastore_id,urls,hasMedia,hashtags,retweet_count,quote_count,user,text,quote_tweet,timestamp,...,retweets_by_cluster_4,quotes_by_cluster_4,author_active_status,retweets_by_suspended,quotes_by_suspended,retweets_crawled,quotes_crawled,retweets_by_l,retweets_by_r,retweets_suspended_ratio
150183,1330487624402935808,[],False,[],26187,1113,25073877,Thank you!,1330185834600026116,2020-11-22T12:25:43Z,...,351,1,suspended,6145,19,24748,83,64,19820,0.248303
343969,1331695288231333890,[],True,[],30251,3790,25073877,https://t.co/axJSsyVFJw,1331684437705302018,2020-11-25T20:24:33Z,...,395,0,suspended,3471,49,17754,178,55,13793,0.195505
366978,1331738816856666114,[],False,[],52821,4724,25073877,“Data group exposes wide spread Mail-In Ballot...,None,2020-11-25T23:17:31Z,...,588,28,suspended,5050,433,29734,3015,87,22945,0.169839
526525,1337745268591259648,[https://twitter.com/i/web/status/133774526859...,False,[],66741,54083,25073877,"I WON THE ELECTION IN A LANDSLIDE, but remembe...",None,2020-12-12T13:05:00Z,...,974,380,suspended,8307,4525,55290,42386,211,43843,0.150244
624540,1333556242984431616,[https://twitter.com/i/web/status/133355624298...,False,[],25653,3721,25073877,"Why is he rushing to put a Democrat in office,...",1333480213255229442,2020-11-30T23:39:19Z,...,285,11,suspended,4268,484,18203,2701,135,14147,0.234467
654123,1333610260381175813,[https://www.breitbart.com/politics/2020/11/30...,False,[],24058,1773,25073877,Donald Trump Calls In to Arizona Voter Fraud H...,None,2020-12-01T03:13:58Z,...,295,15,suspended,3277,182,14398,1045,35,11145,0.227601
702702,1332352538855747584,[https://twitter.com/i/web/status/133235253885...,False,[],91902,27139,25073877,Biden can only enter the White House as Presid...,None,2020-11-27T15:56:13Z,...,1269,191,suspended,7702,1588,53428,17331,267,42901,0.144157
816188,1327750127679889409,[https://twitter.com/i/web/status/132775012767...,False,[],71054,11185,25073877,There is tremendous evidence of wide spread vo...,None,2020-11-14T23:07:53Z,...,1418,99,suspended,9181,1068,55793,9268,224,43876,0.164555
1154189,1329054683441278977,[],False,[],54575,10152,25073877,...AND I WON THE ELECTION. VOTER FRAUD ALL OVE...,1328834297567076354,2020-11-18T13:31:43Z,...,905,61,suspended,6854,872,37124,7035,143,29160,0.184625
1225216,1325432465415163904,[https://twitter.com/i/web/status/132543246541...,False,[],75385,13632,25073877,“We should look at the votes. We’re just begin...,None,2020-11-08T13:38:19Z,...,721,61,suspended,4165,574,26038,4928,168,20932,0.159959


In [41]:
df_recent_tweets.to_pickle("./df_recent_tweets_with_final_metrics.pickle")

In [2]:
df_recent_tweets = pd.read_pickle("./df_recent_tweets_with_final_metrics.pickle")

In [26]:
l_cols = ["retweets_by_cluster_0"]
r_cols = ["retweets_by_cluster_1", "retweets_by_cluster_2", "retweets_by_cluster_3", "retweets_by_cluster_4"]

def get_retweets_suspended_ratio(obj):
    total = obj["retweets_crawled"]
    if total == 0:
        return 0
    else:
        return obj["retweets_by_suspended"] / obj["retweets_crawled"]

df_recent_tweets["retweets_by_l"] = df_recent_tweets[l_cols].sum(axis=1)
df_recent_tweets["retweets_by_r"] = df_recent_tweets[r_cols].sum(axis=1)
df_recent_tweets["retweets_suspended_ratio"] = df_recent_tweets.apply(lambda tweet: get_retweets_suspended_ratio(tweet), axis=1)
df_users_final["retweets_by_l"] = df_users_final[l_cols].sum(axis=1)
df_users_final["retweets_by_r"] = df_users_final[r_cols].sum(axis=1)
df_users_final["retweets_suspended_ratio"] = df_users_final.apply(lambda user: get_retweets_suspended_ratio(user), axis=1)

In [ ]:
df_users_final.loc["25073877"]

In [ ]:
df_recent_tweets[df_recent_tweets["user"] == "25073877"].sum(axis=0)

In [37]:
quote_coverage = df_recent_tweets["quotes_crawled"].sum() / df_recent_tweets["quote_count"].sum()
print(quote_coverage)

retweet_coverage = df_recent_tweets["retweets_crawled"].sum() / df_recent_tweets["retweet_count"].sum()
print(retweet_coverage)

0.6318106978724223
0.6264554896110318


In [40]:
def get_coverage(x, cov_type):
    count = x[cov_type + "_count"]
    if (count == 0):
        return 0
    else:
        return x[cov_type + "s_crawled"] / count

df_recent_tweets["quote_coverage"] = df_recent_tweets.apply(lambda x: get_coverage(x, "quote"), axis=1)
print(df_recent_tweets["quote_coverage"].mean())

df_recent_tweets["retweet_coverage"] = df_recent_tweets.apply(lambda x: get_coverage(x, "retweet"), axis=1)
print(df_recent_tweets["retweet_coverage"].mean())


0.021205824372507363
0.137058125744631


## Top tweets

In [40]:
cols = ["datastore_id", "text", "author_active_status", "user"]

for metric in ["retweets_by_l", "retweets_by_r", "retweets_by_suspended"]:
    print("---Top tweets by {} ---".format(metric))
    i = 1
    for idx, tweet_id, text, active_status, user_id, metric_value in df_recent_tweets.nlargest(10, metric)[cols + [metric]].itertuples():
        user = df_users_final.loc[user_id]
        print("{}) @{} tweeted: {}".format(i, user["handle"], text))
        print("--stats--")
        print("Tweet id:", tweet_id)
        print("User id: {}, active status: {}".format(user["handle"], user_id, active_status))
        print(metric, metric_value)
        print()
        i += 1
    print()
    print()

---Top tweets by retweets_by_l ---
1) @Al_Sanchino tweeted: It’s wild Trump supporters believe there’s voter fraud with 0 evidence but couldn’t believe he sexually assaulted any women with 26 allegations.
--stats--
Tweet id: 1324357146088267777
User id: Al_Sanchino, active status: 1416374742
retweets_by_l 38097

2) @BernieSanders tweeted: In 2016, when Trump won MI, PA and WI by 77, 000 votes, it was part of his "landslide" victory. 

In 2020, when Biden won MI, PA and WI by 255,000 votes, and won the popular vote by 6 million, it was "voter fraud." 

We will not allow Trump to destroy American democracy.
--stats--
Tweet id: 1329968004302598147
User id: BernieSanders, active status: 216776631
retweets_by_l 22402

3) @BernieSanders tweeted: In 2016, when Trump won PA, MI, and WI by a handful of votes, the election system worked just great. Now that he's losing, it's a "fraud" and they're "stealing the election." This is how demagogues destroy faith in democracy and move us toward author

In [5]:
cols = ["datastore_id", "text", "author_active_status", "user", "retweets_crawled", "retweet_count", "retweets_by_suspended"]

for metric in ["retweets_suspended_ratio"]:
    print("---Top tweets (with > 100 retweets) by: {} ---".format(metric))
    i = 1
    for idx, tweet_id, text, active_status, user_id, retweets_crawled, retweet_count, retweets_suspended, metric_value in df_recent_tweets[df_recent_tweets["retweets_crawled"] > 100].nlargest(10, metric)[cols + [metric]].itertuples():
        user = df_users_final.loc[user_id]
        print("{}) @{} tweeted: {}".format(i, user["handle"], text))
        print("--stats--")
        print("Tweet id:", tweet_id)
        print("User id: {}, active status: {}".format(user["handle"], user_id, active_status))
        print("Retweets crawled:", retweets_crawled)
        print("Retweets suspended:", retweets_suspended)
        print(metric, metric_value)
        print()
        i += 1
    print()
    print()

---Top tweets (with > 100 retweets) by: retweets_suspended_ratio ---
1) @PioyCOLX tweeted: 22 different topics on election fraud. Each different, and associated as you will begin to see. The #Truth will get a little darker from here. 100-150 total topics expected + other #Patriots making topics.

https://t.co/BWY8YjyD95
--stats--
Tweet id: 1336818704781348869
User id: PioyCOLX, active status: 995812375432433664
Retweets crawled: 612
Retweets suspended: 525
retweets_suspended_ratio 0.8578431372549019

2) @Godwins1234 tweeted: New show 👇👊👏👏👏
--stats--
Tweet id: 1337847002634317825
User id: Godwins1234, active status: 1293984914652762114
Retweets crawled: 134
Retweets suspended: 106
retweets_suspended_ratio 0.7910447761194029

3) @RaeAnon tweeted: #Election 

💣From 1946 to PRESENT DAY, the US is EXPERT in #ElectionFraud 

💣HOW COULD THIS HAPPEN?

💣LOGIC~You don't pull off an operation like this wo PROFESSIONAL help

💣It's like playing the piano, it takes PRACTICE 

💣CIA has been PRACTICIN

In [6]:
cols = ["datastore_id", "text", "author_active_status", "user"]

for metric in ["retweets_by_l", "retweets_by_r"]:
    print("---Top tweets by {} (SUSPENDED AUTHORS) ---".format(metric))
    i = 1
    for idx, tweet_id, text, active_status, user_id, metric_value in df_recent_tweets[df_recent_tweets["author_active_status"] == "suspended"].nlargest(10, metric)[cols + [metric]].itertuples():
        user = df_users_final.loc[user_id]
        print("{}) @{} tweeted: {}".format(i, user["handle"], text))
        print("Tweet id:", tweet_id)
        print("User id: {}, active status: {}".format(user["handle"], user_id, active_status))
        print(metric, metric_value)
        print()
        i += 1
    print()
    print()

---Top tweets by retweets_by_l (SUSPENDED AUTHORS) ---
1) @20jarett24 tweeted: OMG. Trump's lawyers filed an affidavit claiming statistical irregularities proving election fraud because they *confused Michigan for Minnesota*
They thought "MI" = Minnesota (it's MN), and then compared a bunch of results in Minnesota to Michigan population statistics. 😂😂😂😂
Tweet id: 1329801174279725058
User id: 20jarett24, active status: 1296579598998810624
retweets_by_l 2081

2) @woodymwc tweeted: @jaketapper PLEASE don’t call the Trump Voter Fraud hotline at 1-888-503-3526 and play them the losing sound from The Price Is Right.
Tweet id: 1325250125921443841
User id: woodymwc, active status: 1231252337743859715
retweets_by_l 922

3) @CounselorJeff88 tweeted: I just don't understand why Trump's crack legal team is sitting on a mountain of evidence of election fraud and Biden fraud but keep forgetting to use it when they appear in court.  #DoodieGiuliani
Tweet id: 1330360166450126848
User id: CounselorJeff

## Top users

In [7]:
pd.set_option('expand_frame_repr', False)

cols = ["handle", "active_status", "retweets_by_l", "retweets_by_r", "retweets_by_suspended"]

for metric in ["retweets_by_l", "retweets_by_r", "retweets_by_suspended"]:
    print("---Top users by: {} ---".format(metric))
    i = 1
    print(df_users_final.nlargest(10, metric)[cols])

---Top users by: retweets_by_l ---
                    handle active_status  retweets_by_l  retweets_by_r  retweets_by_suspended
id                                                                                           
32871086      kylegriffin1        active          75264            867                    916
1640929196      mmpadellan        active          73571            731                   1024
255812611       donwinslow        active          69030            651                    924
216776631    BernieSanders        active          60377            282                    421
15952856         AriBerman        active          57713            401                    342
3622368202   JohnFetterman        active          53519            549                    636
341190477   TheRickyDavila        active          50431            594                    733
148529707          RBReich        active          41818            365                    460
22129280        jimsciutt

### Users by suspended ratio

In [9]:
cols = ["handle", "active_status", "retweets_by_l", "retweets_by_r", "retweets_by_suspended", "retweets_suspended_ratio"]

for metric in ["retweets_suspended_ratio"]:
    print("---Top users by: {} ---".format(metric))
    i = 1
    print(df_users_final[df_users_final.retweets_crawled > 1000].nlargest(10, metric)[cols])

---Top users by: retweets_suspended_ratio ---
                              handle active_status  retweets_by_l  retweets_by_r  retweets_by_suspended  retweets_suspended_ratio
id                                                                                                                               
1312752847021375489   ShotGunAnnie2A        active              0           1247                    852                  0.678884
1255946337717301248  honoredpatriot0     suspended              0           1314                    864                  0.653061
971116581131563008   VeteransAlways_     suspended              3           1016                    698                  0.641544
941395744568647680     DontDregMeBro     suspended              1           1132                    736                  0.636128
995812375432433664          PioyCOLX     suspended              0           1450                    955                  0.622962
563997163                     BPPope     sus

In [ ]:
df_recent_tweets_with_shares_by_cluster.info()

## Get top URLs / Youtube channels / Hashtags shared by cluster 

In [58]:
import json
youtube_data = []
with open("./data_export/url_stats/16-dec/youtube_data_details.json") as json_file:
    for line in json_file:
        youtube_data.append(json.loads(line))

youtube_data_by_id = {}
for yt_video in youtube_data:
  youtube_data_by_id[yt_video['id']] = yt_video

In [9]:
len(youtube_data_by_id.keys())

12002

In [59]:
from urllib.parse import urlparse, parse_qs

def video_id(url):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    o = urlparse(url)
    try:
        if o.netloc == 'youtu.be':
            return o.path[1:]
        elif o.netloc in ('www.youtube.com', 'youtube.com'):
            if o.path == '/watch':
                id_index = o.query.index('v=')
                return o.query[id_index+2:id_index+13]
            elif o.path[:7] == '/embed/':
                return o.path.split('/')[2]
            elif o.path[:3] == '/v/':
                return o.path.split('/')[2]
    except:
        pass
    return None

In [33]:
blocked_urls = set()
blocked_domains = set()

for url_data in list(expanded_url_map.values()):
    expanded = url_data["expanded_url"]
    if ("url" in expanded):
        expanded_url = expanded["url"]
        if "google.com/sorry" in expanded_url:
            parsed_url = urlparse(expanded_url)
            blocked_url = parse_qs(parsed_url.query)["continue"][0]
            parse_blocked_url = urlparse(blocked_url)
            blocked_urls.add(blocked_url)
            blocked_domains.add(parse_blocked_url.netloc)

In [44]:
extracted_youtube_ids = 0
youtube_id_with_data = 0

missing_youtube_data_ids = set()
for url in blocked_urls:
    youtube_id = video_id(url)

    if (youtube_id is not None):
        extracted_youtube_ids += 1

        if (youtube_id in youtube_data_by_id):
            youtube_id_with_data += 1
        else:
            missing_youtube_data_ids.add(youtube_id)

print("Blocked urls:", len(blocked_urls))
print("Blocked domains:", ", ".join(blocked_domains))

print("Extracted youtube IDs from blocked urls:", extracted_youtube_ids)
print("Youtube IDs in our Youtube data:", youtube_id_with_data)
print("Missing Youtube Ids:", len(missing_youtube_data_ids))

Blocked urls: 17785
Blocked domains: www.youtube.com, scholar.google.com, youtube.com, music.youtube.com, m.youtube.com
Extracted youtube IDs from blocked urls: 17045
Youtube IDs in our Youtube data: 15086
Missing Youtube Ids: 1647


In [45]:
with open ("./missing_youtube_data_ids.txt", "w") as f:
    for youtube_id in missing_youtube_data_ids:
        f.write(youtube_id + "\n")

In [7]:
import json
with open("./data_export/url_stats/16-dec/expanded_url_map.json") as json_file:
    expanded_url_map = json.load(json_file)

In [14]:
expanded_url_count = 0
youtube_count = 0
youtube_data_count = 0
expanded_youtube_count = 0
expanded_youtube_data_count = 0
google_blocked_expansion = 0
google_blocked_youtube_expansion = 0

def has_youtube_data(url):
    try: 
        youtube_id = video_id(url)
        if youtube_id is not None:
            return True
    except:
        return False


def is_youtube_url(url):
    o = urlparse(url)
    return (o.netloc == 'youtu.be' or o.netloc in ('www.youtube.com', 'youtube.com'))

for key, val in expanded_url_map.items():
    if (is_youtube_url(key)):
        youtube_count += 1
    if (has_youtube_data(key)):
        youtube_data_count += 1
    
    if ("url" in val["expanded_url"]):
        expanded_url_count += 1
        expanded_url = val["expanded_url"]["url"]
        
        if (is_youtube_url(expanded_url)):
            expanded_youtube_count += 1
        if (has_youtube_data(expanded_url)):
            expanded_youtube_data_count += 1
        if ("google.com/sorry" in expanded_url):
            google_blocked_expansion += 1
            if ("youtube" in expanded_url):
                google_blocked_youtube_expansion += 1

print("--Unexpanded urls")
print("Total urls", len(expanded_url_map))
print("Youtube domain count", youtube_count)
print("Youtube data count", youtube_data_count)
print()
print("--Expanded urls")
print("Total urls", expanded_url_count)
print("Youtube domain count", expanded_youtube_count)
print("Youtube data count", expanded_youtube_data_count)
print("Google blocked expansion", google_blocked_expansion)
print("Google blocked Youtube expansion", google_blocked_youtube_expansion)

--Unexpanded urls
Total urls 155064
Youtube domain count 17477
Youtube data count 17322

--Expanded urls
Total urls 151830
Youtube domain count 223
Youtube data count 66
Google blocked expansion 18349
Google blocked Youtube expansion 18345


In [11]:
def add_hashtag(hashtag_map, hashtag, tweet_id, retweet_count, quote_count):
    if (hashtag in hashtag_map):
        hashtag_map[hashtag]["retweet_count"] += retweet_count
        hashtag_map[hashtag]["quote_count"] += quote_count
        hashtag_map[hashtag]["tweet_count"] += 1
        #hashtag_map[hashtag]["tweet_ids"].append(tweet_id)
    else:
        hashtag_map[hashtag] = {
            "retweet_count": retweet_count,
            "quote_count": quote_count,
            "tweet_count": 1,
            #"tweet_ids": [tweet_id]
        }

def add_yt_video(youtube_map, yt_data, tweet_id, retweet_count, quote_count):
    if (yt_data["id"] in youtube_map):
        youtube_map[yt_data["id"]]["retweet_count"] += retweet_count
        youtube_map[yt_data["id"]]["quote_count"] += quote_count
        youtube_map[yt_data["id"]]["tweet_count"] += 1
        #youtube_map[yt_data["id"]]["tweet_ids"].append(tweet_id)
    else:
        youtube_map[yt_data["id"]] = {
            "video_title": yt_data["snippet"]["title"],
            "video_description": yt_data["snippet"]["description"],
            "channel_title": yt_data["snippet"]["channelTitle"],
            "published_at": yt_data["snippet"]["publishedAt"],
            "retweet_count": retweet_count,
            "quote_count": quote_count,
            "tweet_count": 1,
            #"tweet_ids": [tweet_id]
        }

def add_url(url_map, url, domain, tweet_id, retweet_count, quote_count):
    if (url in url_map):
        url_map[url]["retweet_count"] += retweet_count
        url_map[url]["quote_count"] += quote_count
        url_map[url]["tweet_count"] += 1
        #url_map[url]["tweet_ids"].append(tweet_id)
    else:
        url_map[url] = {
            "domain": domain,
            "retweet_count": retweet_count,
            "quote_count": quote_count,
            "tweet_count": 1,
            #"tweet_ids": [tweet_id]
        }

def get_video_id(url):
    try:
        return video_id(url)
    except:
        return None

def handle_url(cluster_map, url, tweet_id, retweet_count, quote_count):
    expanded_url_data = expanded_url_map[url]["expanded_url"]
    expanded_url = expanded_url_data["url"] if "url" in expanded_url_data else url
    domain = expanded_url_data["domain"]["domain"] if "url" in expanded_url_data else "URL NOT AVAILABLE"
    
    youtube_id = get_video_id(expanded_url) or get_video_id(url)

    if (youtube_id in youtube_data_by_id):
        add_yt_video(cluster_map["youtube"], youtube_data_by_id[youtube_id], tweet_id, retweet_count, quote_count)
    else:
        add_url(cluster_map["url"], expanded_url, domain, tweet_id, retweet_count, quote_count)


In [10]:
df_recent_tweets.columns

Index(['datastore_id', 'urls', 'hasMedia', 'hashtags', 'retweet_count',
       'quote_count', 'user', 'text', 'quote_tweet', 'timestamp', 'tokens',
       'election fraud', 'voter fraud', '#voterfraud', '#stopthesteal',
       '#ballotharvesting', 'ballot fraud', '#electionfraud',
       '#electioninterference', 'ballot harvesting', 'election interference',
       '#electiontampering', '#cheatingdemocrats', 'election tampering',
       'democrats cheat', '#voterfraudisreal', 'cheating democrats',
       '#gopvoterfraud', 'destroyed ballots', 'stolen ballots', '#ballotfraud',
       'discarded ballots', 'hacked voting machine', 'pre-filled ballot',
       'harvest ballot', '#stopvoterfraud', '#democratvoterfraud',
       '#ballotvoterfraud', '#nomailinvoting', '#ilhanomarballotharvesting',
       'vote by mail fraud', '#mailinvoterfraud', '#votebymailfraud',
       '#ilhanomarvoterfraud', '#stopgopvoterfraud', '#discardedballots',
       '#hackedvotingmachines', 'cluster', 'retweets_by_

In [3]:
retweets_by_cluster_cols = ["retweets_by_cluster_" + str(c) for c in range(5)]
quotes_by_cluster_cols = ["quotes_by_cluster_" + str(c) for c in range(5)]
tweets_by_cluster_cols = ["tweets_by_cluster_" + str(c) for c in range(5)]
extra_cols = ["retweets_by_suspended", "quotes_by_suspended", "retweets_crawled", "quotes_crawled"]

def get_inner_dict ():
    x = {
        "tweet_count": 0,
        "quote_count": 0,
        "retweet_count": 0,
        "retweets_by_suspended": 0,
        "quotes_by_suspended": 0,
        "tweets_by_suspended": 0,
        "quotes_crawled": 0,
        "retweets_crawled": 0
    }
    for col in retweets_by_cluster_cols + quotes_by_cluster_cols + tweets_by_cluster_cols:
        x[col] = 0
    return x

In [29]:
df_recent_tweets["cluster"][:15]

0        0
1        1
2        0
3        0
4        2
5        0
6        2
7        0
8        0
9        0
10       2
11       2
12    <NA>
13    <NA>
14       1
Name: cluster, dtype: Int64

In [5]:
from collections import defaultdict

content_shares_by_cluster = {
    "hashtag": defaultdict(lambda: get_inner_dict()),
    "url": defaultdict(lambda: get_inner_dict()),
    "media_by_tweet_id": defaultdict(lambda: get_inner_dict())
}

relevant_cols = retweets_by_cluster_cols + quotes_by_cluster_cols + extra_cols

for i, tweet in df_recent_tweets.iterrows():
    str_tweet_cluster = pd.notna(tweet["cluster"]) and str(tweet["cluster"])
    quote_count = tweet["quote_count"]
    retweet_count = tweet["retweet_count"]

    is_suspended_user = tweet.author_active_status == "suspended"
    
    for hashtag in tweet["hashtags"]:
        lowered_hashtag = hashtag.lower()
        for col in relevant_cols:
            content_shares_by_cluster["hashtag"][lowered_hashtag][col] += tweet[col]
        content_shares_by_cluster["hashtag"][lowered_hashtag]["tweet_count"] += 1
        content_shares_by_cluster["hashtag"][lowered_hashtag]["quote_count"] += quote_count
        content_shares_by_cluster["hashtag"][lowered_hashtag]["retweet_count"] += retweet_count
        if str_tweet_cluster:
            content_shares_by_cluster["hashtag"][lowered_hashtag]["tweets_by_cluster_" + str_tweet_cluster] += 1
        if is_suspended_user:
            content_shares_by_cluster["hashtag"][lowered_hashtag]["tweets_by_suspended"] += 1
    for url in tweet["urls"]:
        if "twitter.com/" not in url:
            for col in relevant_cols:
                content_shares_by_cluster["url"][url][col] += tweet[col]
            content_shares_by_cluster["url"][url]["tweet_count"] += 1
            content_shares_by_cluster["url"][url]["quote_count"] += quote_count
            content_shares_by_cluster["url"][url]["retweet_count"] += retweet_count
            if str_tweet_cluster:
                content_shares_by_cluster["url"][url]["tweets_by_cluster_" + str_tweet_cluster] += 1
            if is_suspended_user:
                content_shares_by_cluster["url"][url]["tweets_by_suspended"] += 1

    if (tweet["hasMedia"]):
        content_shares_by_cluster["media_by_tweet_id"][tweet["datastore_id"]]["tweet_count"] += 1
        content_shares_by_cluster["media_by_tweet_id"][tweet["datastore_id"]]["quote_count"] = quote_count
        content_shares_by_cluster["media_by_tweet_id"][tweet["datastore_id"]]["retweet_count"] = retweet_count
        for col in relevant_cols:
            content_shares_by_cluster["media_by_tweet_id"][tweet["datastore_id"]][col] = tweet[col]

        if str_tweet_cluster:
            content_shares_by_cluster["hashtag"][lowered_hashtag]["tweets_by_cluster_" + str_tweet_cluster] += 1
        
        if is_suspended_user:
            content_shares_by_cluster["media_by_tweet_id"][tweet["datastore_id"]]["tweets_by_suspended"] += 1
    if (i % 100000 == 0):
        print("Processed", i)

Processed 0
Processed 100000
Processed 200000
Processed 300000
Processed 400000
Processed 500000
Processed 600000
Processed 700000
Processed 800000
Processed 900000
Processed 1000000
Processed 1100000
Processed 1200000
Processed 1300000
Processed 1400000
Processed 1500000
Processed 1600000
Processed 1700000
Processed 1800000
Processed 1900000
Processed 2000000
Processed 2100000
Processed 2200000
Processed 2300000
Processed 2400000
Processed 2500000
Processed 2600000
Processed 2700000
Processed 2800000
Processed 2900000
Processed 3000000
Processed 3100000
Processed 3200000
Processed 3300000
Processed 3400000
Processed 3500000
Processed 3600000
Processed 3700000
Processed 3800000
Processed 3900000
Processed 4000000
Processed 4100000
Processed 4200000
Processed 4300000
Processed 4400000
Processed 4500000
Processed 4600000
Processed 4700000
Processed 4800000
Processed 4900000
Processed 5000000
Processed 5100000
Processed 5200000
Processed 5300000
Processed 5400000
Processed 5500000
Process

In [6]:
print(len(content_shares_by_cluster["media_by_tweet_id"]))
print(len(content_shares_by_cluster["url"]))
print(content_shares_by_cluster["hashtag"]["concedenowtrump"])

178773
155064
{'tweet_count': 1049, 'quote_count': 64, 'retweet_count': 538, 'retweets_by_suspended': 11, 'quotes_by_suspended': 4, 'tweets_by_suspended': 17, 'quotes_crawled': 61, 'retweets_crawled': 455, 'retweets_by_cluster_0': 425, 'retweets_by_cluster_1': 0, 'retweets_by_cluster_2': 27, 'retweets_by_cluster_3': 0, 'retweets_by_cluster_4': 0, 'quotes_by_cluster_0': 39, 'quotes_by_cluster_1': 0, 'quotes_by_cluster_2': 5, 'quotes_by_cluster_3': 0, 'quotes_by_cluster_4': 0, 'tweets_by_cluster_0': 642, 'tweets_by_cluster_1': 4, 'tweets_by_cluster_2': 95, 'tweets_by_cluster_3': 0, 'tweets_by_cluster_4': 0}


In [10]:
import pickle 

if ("hashtag" in content_shares_by_cluster):
    with open("./content_shares_by_cluster.pickle", "wb") as f:
        pickle.dump({
            "hashtag": dict(content_shares_by_cluster["hashtag"]),
            "url": dict(content_shares_by_cluster["url"]),
            "media_by_tweet_id": dict(content_shares_by_cluster["media_by_tweet_id"])
        }, f)

In [1]:
import pickle 
with open("./content_shares_by_cluster.pickle", "rb") as f:
    test = pickle.load(f)

In [3]:
import pandas as pd
df_url = pd.DataFrame.from_dict(test["url"]).T

In [55]:
df_hashtag = pd.DataFrame.from_dict(content_shares_by_cluster["hashtag"]).T
df_url = pd.DataFrame.from_dict(urls_without_twitter).T

In [39]:
(df_url.tweets_cluster_0 > 0).sum()

AttributeError: 'DataFrame' object has no attribute 'tweets_cluster_0'

In [4]:
df_url.head()

,tweet_count,quote_count,retweet_count,retweets_by_suspended,quotes_by_suspended,tweets_by_suspended,quotes_crawled,retweets_crawled,retweets_by_cluster_0,retweets_by_cluster_1,...,quotes_by_cluster_0,quotes_by_cluster_1,quotes_by_cluster_2,quotes_by_cluster_3,quotes_by_cluster_4,tweets_by_cluster_0,tweets_by_cluster_1,tweets_by_cluster_2,tweets_by_cluster_3,tweets_by_cluster_4
https://www.bbc.co.uk/news/election-us-2020-54926084,358,2,55,1,0,6,9,56,54,0,...,6,0,0,0,0,179,1,16,0,0
https://ouo.io/DgaKIL,20,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,7,4,0,0,0
https://youtu.be/2lKZ3W5qTjg,133,21,380,63,1,23,11,238,0,20,...,0,1,9,0,0,0,22,83,0,0
https://newshourfirst.com/2020/11/12/emails-reveal-deep-election-fraud-starting-back-as-far-as-obama/,21,33,331,90,11,3,24,218,0,1,...,0,3,21,0,0,0,1,16,0,0
https://youtu.be/p25v2q_b8bQ,144,7,89,15,4,19,9,52,0,0,...,0,2,5,0,0,3,12,93,0,0


In [21]:
df_url["tweets_by_promoters"] = df_url.apply(lambda url: url["tweets_by_cluster_1"] + url["tweets_by_cluster_2"] + url["tweets_by_cluster_3"] + url["tweets_by_cluster_4"], axis=1)
df_url["retweets_by_promoters"] = df_url.apply(lambda url: url["retweets_by_cluster_1"] + url["retweets_by_cluster_2"] + url["retweets_by_cluster_3"] + url["retweets_by_cluster_4"], axis=1)

In [32]:
from collections import defaultdict
domain_shares = defaultdict(lambda: {
    "tweets_by_promoters": 0,
    "retweets_by_promoters": 0,
})

for url, tweets_by_promoters, retweets_by_promoters in df_url[["tweets_by_promoters", "retweets_by_promoters"]].itertuples():
    if url in expanded_url_map:
        try:
            domain = expanded_url_map[url]["expanded_url"]["domain"]["domain"]
            domain_shares[domain]["tweets_by_promoters"] += tweets_by_promoters
            domain_shares[domain]["retweets_by_promoters"] += retweets_by_promoters
        except:
            pass

In [36]:
df_domains = pd.DataFrame.from_dict(domain_shares).T
df_domains.nlargest(20, ["retweets_by_promoters"])

,tweets_by_promoters,retweets_by_promoters
breitbart,8818,108432
pscp,2241,83325
google,64514,78953
thegatewaypundit,20252,65984
theepochtimes,15002,56932
justthenews,3027,54653
thefederalist,4528,42025
whitehouse,19319,40024
oann,3283,38349
foxnews,5303,34471


In [19]:
len(expanded_url_map)
expanded_url_map["https://rumble.com/vatz4r-joe-biden-admits-to-voter-fraud.html"]["expanded_url"]["domain"]

{'subdomain': '', 'domain': 'rumble', 'suffix': 'com'}

In [297]:
df_recent_tweets["cluster"].value_counts(dropna=False)

2      3982990
NaN    1724417
0      1199587
1       644219
3        27699
4        24191
Name: cluster, dtype: Int64

In [72]:
df_recent_tweets["retweet_count"] = df_recent_tweets["retweet_count"].astype('Int64')
df_recent_tweets["quote_count"] = df_recent_tweets["quote_count"].astype('Int64')
df_recent_tweets["quote_tweet"] = df_recent_tweets["quote_count"].astype(str)

In [81]:
def get_url(tweet_id):
    if tweet_id in tweet_id_to_url:
        return tweet_id_to_url[tweet_id][0]

def get_domain(tweet_id):
    if tweet_id in tweet_id_to_url:
        return tweet_id_to_url[tweet_id][1]

df_recent_tweets["expanded_url"] = df_recent_tweets.apply(lambda tweet: get_url(tweet["datastore_id"]), axis=1)
df_recent_tweets["expanded_url_domain"] = df_recent_tweets.apply(lambda tweet: get_domain(tweet["datastore_id"]), axis=1)

In [ ]:
df_recent_tweets

In [104]:
tweet_export = df_recent_tweets.rename(columns={'datastore_id': 'tweet_id', 'user': 'user_id'})[[
    "tweet_id",
    "user_id", 
    "cluster", 
    "retweet_count", 
    "quote_count", 
    "quote_tweet"
]]

In [105]:
tweet_export.head()
tweet_export.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7603103 entries, 0 to 7609004
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   tweet_id       object
 1   user_id        object
 2   cluster        Int64 
 3   retweet_count  Int64 
 4   quote_count    Int64 
 5   quote_tweet    object
dtypes: Int64(3), object(3)
memory usage: 747.8+ MB


In [108]:
tweet_export.to_csv("./tweets_with_cluster.csv", index=False)

In [107]:
pd.read_csv("./tweets_with_cluster.csv")

,tweet_id,user_id,cluster,retweet_count,quote_count,quote_tweet
0,1327172028709801985,2313341894,0,0,0,NaN
1,1327172032002486273,750121838,1,0,0,NaN
2,1327172032061173761,546655747,0,0,0,1.326904e+18
3,1327172033143316480,25709850,0,0,0,NaN
4,1327172039669665794,1254057289113317382,2,0,0,NaN
